In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install git+https://github.com/allenai/longformer.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/allenai/longformer.git to /tmp/pip-req-build-alvz6yqs
  Running command git clone --filter=blob:none --quiet https://github.com/allenai/longformer.git /tmp/pip-req-build-alvz6yqs
  Resolved https://github.com/allenai/longformer.git to commit caefee668e39cacdece7dd603a0bebf24df6d8ca
  Preparing metadata (setup.py) ... done
  Cloning http://github.com/ibeltagy/transformers.git (to revision longformer_encoder_decoder) to /tmp/pip-install-e_5xx_4_/transformers_e32e857f2cf645079e272adfaee11518
  Running command git clone --filter=blob:none --quiet http://github.com/ibeltagy/transformers.git /tmp/pip-install-e_5xx_4_/transformers_e32e857f2cf645079e272adfaee11518
  Running command git checkout -b longformer_encoder_decoder --track origin/longformer_encoder_decoder
  Switched to a new branch 'longformer_encoder_decoder'
  Branch 'longformer_encoder_decoder' set up to 

In [ ]:
import os
import re
import random
import pandas as pd
import numpy as np
import csv
import tensorflow as tf
import torch
from google.colab import drive
import textwrap
import progressbar
import keras
from keras_preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
import time
import datetime
import json
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import time
import datetime
import random

from transformers import LongformerTokenizerFast, LongformerForSequenceClassification, Trainer, TrainingArguments, LongformerConfig
import torch.nn as nn
import torch

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


def metrics_calculator(preds, test_labels):
    cm = confusion_matrix(test_labels, preds)
    TP = []
    FP = []
    FN = []
    for i in range(0,2):
        summ = 0
        for j in range(0,2):
            if(i!=j):
                summ=summ+cm[i][j]

        FN.append(summ)
    for i in range(0,2):
        summ = 0
        for j in range(0,2):
            if(i!=j):
                summ=summ+cm[j][i]

        FP.append(summ)
    for i in range(0,2):
        TP.append(cm[i][i])
    precision = []
    recall = []
    for i in range(0,2):
        precision.append(TP[i]/(TP[i] + FP[i]))
        recall.append(TP[i]/(TP[i] + FN[i]))

    macro_precision = sum(precision)/2
    macro_recall = sum(recall)/2
    micro_precision = sum(TP)/(sum(TP) + sum(FP))
    micro_recall = sum(TP)/(sum(TP) + sum(FN))
    micro_f1 = (2*micro_precision*micro_recall)/(micro_precision + micro_recall)
    macro_f1 = (2*macro_precision*macro_recall)/(macro_precision + macro_recall)
    return macro_precision, macro_recall, macro_f1, micro_precision, micro_recall, micro_f1

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
input_file = '/content/drive/MyDrive/Colab Notebooks/SemEval2023/CJPE-main/Data/trainData/sample-train.csv'
#pred_file = '/content/drive/MyDrive/Colab Notebooks/SemEval2023/CJPE-main/Data/trainData/ILDC_single_dev_pred.csv'

#save the trained model
output_dir     = '/content/drive/MyDrive/Colab Notebooks/SemEval2023/CJPE-main/Models/finetuned/longformer_trained_on_single_preprocessed_2_fixed/'
pretrained_dir = None


#test_file  = '/content/drive/MyDrive/Colab Notebooks/SemEval2023/CJPE-main/Data/trainData/ILDC_single_test_preprocessed_2.csv'
#pred_file  = '/content/drive/MyDrive/Colab Notebooks/SemEval2023/CJPE-main/Data/trainData/ILDC_single_test_preprocessed_2_preds.csv'

#load data
df = pd.read_csv(input_file) # path to multi_dataset
df = df.applymap(lambda x: x.strip().replace('-\n', '').replace('\n', '') if isinstance(x, str) else x)
train_set = df.query(" split=='train' ")
validation_set = df.query(" split=='dev' ")
#validation_set = pd.read_csv(test_file)
#test_set = pd.read_csv(test_file)

validation_set['id'] = range(len(validation_set))

<ipython-input-4-760f0f696c56>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validation_set['id'] = range(len(validation_set))


# Building the model

In [ ]:
model = LongformerForSequenceClassification.from_pretrained('allenai/longformer-base-4096', gradient_checkpointing=False, attention_window = 512)
#model = LongformerForSequenceClassification.from_pretrained(pretrained_dir)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

Downloading:   0%|          | 0.00/694 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/597M [00:00<?, ?B/s]

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', '

LongformerForSequenceClassification(
  (longformer): LongformerModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(4098, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): LongformerEncoder(
      (layer): ModuleList(
        (0): LongformerLayer(
          (attention): LongformerAttention(
            (self): LongformerSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (query_global): Linear(in_features=768, out_features=768, bias=True)
              (key_global): Linear(in_features=768, out_features=768, bias=True)
              (value_

# Preparing data

In [ ]:
tokenizer = LongformerTokenizerFast.from_pretrained('allenai/longformer-base-4096')


def input_id_maker(dataf, tokenizer):
  input_ids = []
  lengths = []

  for i in progressbar.progressbar(range(len(dataf['text']))):
    sen = dataf['text'].iloc[i]
    sen = tokenizer(sen)
    #CLS = tokenizer.cls_token
    #SEP = tokenizer.sep_token
    #sen = [CLS] + sen + [SEP]
    #encoded_sent = tokenizer.convert_tokens_to_ids(sen)
    input_ids.append(sen['input_ids'])
    lengths.append(len(sen['input_ids']))

  input_ids = pad_sequences(input_ids, maxlen=1024, value=0, dtype="long", truncating="pre", padding="post")
  return input_ids, lengths

train_input_ids, train_lengths = input_id_maker(train_set, tokenizer)
validation_input_ids, validation_lengths = input_id_maker(validation_set, tokenizer)

def att_masking(input_ids):
  attention_masks = []
  for sent in input_ids:
    att_mask = [int(token_id > 0) for token_id in sent]
    attention_masks.append(att_mask)
  return attention_masks


train_attention_masks = att_masking(train_input_ids)
validation_attention_masks = att_masking(validation_input_ids)

train_labels = train_set['label'].to_numpy().astype('int')
validation_labels = validation_set['label'].to_numpy().astype('int')
#validation_labels = validation_set['id'].to_numpy().astype('int')

train_inputs = train_input_ids
validation_inputs = validation_input_ids
train_masks = train_attention_masks
validation_masks = validation_attention_masks

train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

# max batch size should be 6 due to colab limits
batch_size = 6
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size = batch_size)
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size = batch_size)

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

100% (4982 of 4982) |####################| Elapsed Time: 0:00:31 Time:  0:00:31
100% (994 of 994) |######################| Elapsed Time: 0:00:07 Time:  0:00:07


# Evaluation of the pretrained model on training set before fine-tuning 

In [ ]:
'''batch_size = 6
prediction_data = TensorDataset(train_inputs, train_masks, train_labels)
prediction_sampler = SequentialSampler(train_data)
prediction_dataloader = DataLoader(train_data, shuffle=False, sampler=None, batch_size = batch_size)
print('Predicting labels for {:,} test sentences...'.format(len(prediction_data)))
model.eval()
predictions , true_labels = [], []
explanations = []
for (step, batch) in enumerate(prediction_dataloader):
  batch = tuple(t.to(device) for t in batch)  
  b_input_ids, b_input_mask, b_labels = batch
  with torch.no_grad():
      outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
      logits= outputs[0]
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  predictions.append(logits)
  true_labels.append(label_ids)
predictions = np.concatenate(predictions, axis=0)
true_labels = np.concatenate(true_labels, axis=0)
pred_flat = np.argmax(predictions, axis=1).flatten()
labels_flat = true_labels.flatten()
print(flat_accuracy(predictions,true_labels))
macro_precision, macro_recall, macro_f1, micro_precision, micro_recall, micro_f1 = metrics_calculator(pred_flat, labels_flat)
print(macro_precision, macro_recall, macro_f1, micro_precision, micro_recall, micro_f1)
print('    DONE.')
print(predictions)'''

"batch_size = 6\nprediction_data = TensorDataset(train_inputs, train_masks, train_labels)\nprediction_sampler = SequentialSampler(train_data)\nprediction_dataloader = DataLoader(train_data, shuffle=False, sampler=None, batch_size = batch_size)\nprint('Predicting labels for {:,} test sentences...'.format(len(prediction_data)))\nmodel.eval()\npredictions , true_labels = [], []\nexplanations = []\nfor (step, batch) in enumerate(prediction_dataloader):\n  batch = tuple(t.to(device) for t in batch)  \n  b_input_ids, b_input_mask, b_labels = batch\n  with torch.no_grad():\n      outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)\n      logits= outputs[0]\n  logits = logits.detach().cpu().numpy()\n  label_ids = b_labels.to('cpu').numpy()\n  predictions.append(logits)\n  true_labels.append(label_ids)\npredictions = np.concatenate(predictions, axis=0)\ntrue_labels = np.concatenate(true_labels, axis=0)\npred_flat = np.argmax(predictions, axis=1).flatten()\nlabels_flat

# Training

In [ ]:
lr = 2e-6
max_grad_norm = 1.0
epochs = 10
num_total_steps = len(train_dataloader)*epochs
num_warmup_steps = 1000
warmup_proportion = float(num_warmup_steps) / float(num_total_steps)  # 0.1
optimizer = AdamW(model.parameters(), lr=lr, correct_bias=True)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = num_warmup_steps, num_training_steps = num_total_steps)
seed_val = 2212
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

loss_values = []

# For each epoch...
for epoch_i in range(0, epochs):
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):
        if step % 40 == 0 and not step == 0:
            print('  Batch {:>5,}  of  {:>5,}. '.format(step, len(train_dataloader)))

        
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()        

        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        
        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader)            
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))

    # Create output directory if needed
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    print("  Saving model to %s (epoch %d)" % (output_dir,epoch_i))

    # Save a trained model, configuration and tokenizer using `save_pretrained()`.
    # They can then be reloaded using `from_pretrained()`
    model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
    model_to_save.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)


    if (epoch_i+1)%5 == 0:
        
      print("")
      print("Running Validation...")

      t0 = time.time()

      model.eval()

      eval_loss, eval_accuracy = 0, 0
      nb_eval_steps, nb_eval_examples = 0, 0

      for batch in validation_dataloader:
          batch = tuple(t.to(device) for t in batch)
          b_input_ids, b_input_mask, b_labels = batch
          
          with torch.no_grad():        
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
      
          logits = outputs[0]

          logits = logits.detach().cpu().numpy()
          label_ids = b_labels.to('cpu').numpy()
          
          tmp_eval_accuracy = flat_accuracy(logits, label_ids)
          eval_accuracy += tmp_eval_accuracy

          nb_eval_steps += 1

      # Report the final accuracy for this validation run.
      print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))

print("")
print("Training complete!")

model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

======== Epoch 1 / 10 ========
Training...
  Batch    40  of    831. 
  Batch    80  of    831. 
  Batch   120  of    831. 
  Batch   160  of    831. 
  Batch   200  of    831. 
  Batch   240  of    831. 
  Batch   280  of    831. 
  Batch   320  of    831. 
  Batch   360  of    831. 
  Batch   400  of    831. 
  Batch   440  of    831. 
  Batch   480  of    831. 
  Batch   520  of    831. 
  Batch   560  of    831. 
  Batch   600  of    831. 
  Batch   640  of    831. 
  Batch   680  of    831. 
  Batch   720  of    831. 
  Batch   760  of    831. 
  Batch   800  of    831. 

  Average training loss: 0.68
  Saving model to /content/drive/MyDrive/Colab Notebooks/SemEval2023/CJPE-main/Models/finetuned/longformer_trained_on_single_preprocessed_2_fixed/ (epoch 0)
======== Epoch 2 / 10 ========
Training...
  Batch    40  of    831. 
  Batch    80  of    831. 
  Batch   120  of    831. 
  Batch   160  of    831. 
  Batch   200  of    831. 
  Batch   240  of    831. 
  Batch   280  of    831

#Predicting

## Evaluation on training set

In [ ]:
batch_size = 6

prediction_data = TensorDataset(train_inputs, train_masks, train_labels)
prediction_sampler = SequentialSampler(train_data)
prediction_dataloader = DataLoader(train_data, shuffle=False, sampler=None, batch_size = batch_size)

print('Predicting labels for {:,} test sentences...'.format(len(prediction_data)))
model.eval()

predictions , true_labels = [], []
explanations = []

for (step, batch) in enumerate(prediction_dataloader):
  batch = tuple(t.to(device) for t in batch)
  
  b_input_ids, b_input_mask, b_labels = batch
  #print(b_input_ids)

  with torch.no_grad():
      outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

      #print(outputs)
      logits= outputs[0]
      #attentions = outputs.attentions[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  #attentions = attentions.detach().cpu().numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

predictions = np.concatenate(predictions, axis=0)
true_labels = np.concatenate(true_labels, axis=0)
pred_flat = np.argmax(predictions, axis=1).flatten()
labels_flat = true_labels.flatten()

print(flat_accuracy(predictions,true_labels))

macro_precision, macro_recall, macro_f1, micro_precision, micro_recall, micro_f1 = metrics_calculator(pred_flat, labels_flat)
print(macro_precision, macro_recall, macro_f1, micro_precision, micro_recall, micro_f1)

print('    DONE.')

**Original** 

3 epochs
Predicting labels for 4,982 test sentences...
0.8855881172219991
0.8914680269703594 0.8649874107904924 0.878028105779464 
0.8855881172219991 0.8855881172219991 0.8855881172219991


10 epochs
Predicting labels for 4,982 test sentences...
0.9353673223604978
0.939945656855312 0.9231056446561747 0.9314495428110406 0.9353673223604978 0.9353673223604978 0.9353673223604978


**Preprocessed_2**

10 epochs 
Predicting labels for 4,982 test sentences...
0.9283420313127259
0.9358543523405729 0.9129802171516941 0.924275783156626 0.9283420313127259 0.9283420313127259 0.9283420313127259

##Evaluation on dev set

In [ ]:
batch_size = 6

prediction_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
prediction_sampler = SequentialSampler(validation_data)
prediction_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size = batch_size)

print('Predicting labels for {:,} test sentences...'.format(len(prediction_data)))
model.eval()

predictions , true_labels = [], []
explanations = []

for (step, batch) in enumerate(prediction_dataloader):
  batch = tuple(t.to(device) for t in batch)
  
  b_input_ids, b_input_mask, b_labels = batch
  #print(b_input_ids)

  with torch.no_grad():
      outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

      #print(outputs)
      logits= outputs[0]
      #attentions = outputs.attentions[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  #attentions = attentions.detach().cpu().numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

predictions = np.concatenate(predictions, axis=0)
true_labels = np.concatenate(true_labels, axis=0)
pred_flat = np.argmax(predictions, axis=1).flatten()
labels_flat = true_labels.flatten()

print(flat_accuracy(predictions,true_labels))

macro_precision, macro_recall, macro_f1, micro_precision, micro_recall, micro_f1 = metrics_calculator(pred_flat, labels_flat)
print(macro_precision, macro_recall, macro_f1, micro_precision, micro_recall, micro_f1)

print('    DONE.')

**Original**

3 epochs
Predicting labels for 994 test sentences...
0.5925553319919518
0.7062463914549654 0.5925553319919517 0.6444248685566497 
0.5925553319919518 0.5925553319919518 0.5925553319919518


10 epochs 
Predicting labels for 994 test sentences...
0.6247484909456741
0.6833642768732744 0.624748490945674 0.6527431139665323 0.6247484909456741 0.6247484909456741 0.6247484909456741

**preprocessed_2**

10epochs 
Predicting labels for 994 test sentences...
0.6187122736418511
0.6935153898949369 0.6187122736418511 0.6539817679670216 0.6187122736418511 0.6187122736418511 0.6187122736418511

#Predict testset

In [ ]:
'''batch_size = 1

# Create the DataLoader.
prediction_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, shuffle=False, sampler=None, batch_size = batch_size)
assert len(prediction_data) == 1500

print('Predicting labels for {:,} test sentences...'.format(len(prediction_data)))
model.eval()

predictions , true_labels = [], []
explanations = []

for (step, batch) in enumerate(prediction_dataloader):
  batch = tuple(t.to(device) for t in batch)
  
  b_input_ids, b_input_mask, b_labels = batch
  #print(b_input_ids)

  with torch.no_grad():
      outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

      logits = outputs[0]
      
  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

predictions = np.concatenate(predictions, axis=0)
true_labels = np.concatenate(true_labels, axis=0)
pred_flat = np.argmax(predictions, axis=1).flatten()
labels_flat = true_labels.flatten()


print('    DONE.')

validation_set['id2'] = labels_flat
validation_set['prediction'] = pred_flat
assert all(validation_set['id2'] == validation_set['id'])
validation_set.to_csv(pred_file, index=False,)'''